Run electronic structure calulations of P3HT with gaussian

In [1]:
import os,sys

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


In [5]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Setup a Gaussian calculation of a single p3ht monomer built in the P3HT_build example

In [6]:
p3ht_n1 = buildingblock.Container('thiophene_hexane')
p3ht_n1.read_cply()

In [7]:
print p3ht_n1.n_particles

27


In [8]:
gaus_i = gaussian.Gaussian('p3ht_n1_gaus')

In [9]:
gaus_i.load_json()

In [10]:
gaus_i.set_strucC(p3ht_n1)

In [11]:
gaus_i.set_resource(peregrine)

In [12]:
gaus_i.make_dir()

In [13]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run gaussian_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/gaussian_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n1_gaus/gaussian_peregrine.pbs 


In [14]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates com gaussian.com templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/gaussian.com to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n1_gaus/gaussian.com 


In [15]:
os.chdir(gaus_i.dir['launch'])

In [16]:
gaus_i.load_str('templates','com')

In [17]:
gaus_i.load_str('templates','run')

In [18]:
gaus_i.properties['commands'] = 'HF/3-21G SP Pop=MK nosym'
gaus_i.properties['charge'] = 0
gaus_i.properties['spin_mult'] = 1
gaus_i.properties['coord'] = gaus_i.strucC.write_coord()

In [19]:
gaus_i.replacewrite_prop('com','input','com','%s.com'%(gaus_i.tag))

In [20]:
gaus_i.properties['input_com'] = gaus_i.files['input']['com']

In [21]:
gaus_i.properties['scratch_dir'] = gaus_i.dir['scratch']

In [22]:
gaus_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaus_i.tag))

In [23]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

In [24]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

In [25]:
print gaus_i.dir['home'] 

/Users/tkemper/Development/streamm_ext/streamm-tools/examples


In [26]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()
os.chdir(gaus_i.dir['launch'])

In [27]:
gaus_i.meta['status'] = 'written'

In [ ]:
gaus_i.push()

In [ ]:
gaus_i.run()

In [28]:
gaus_i.check()
print gaus_i.meta['status']

finished


In [29]:
gaus_i.analysis()

Set charges to ESP charges from fchk analysis

In [30]:
gaus_i.strucC.properties['charge'] = 0.0 
for p,pt in gaus_i.strucC.particles.iteritems(): # properties["charge"] 
    pt.properties["charge"] = round(pt.properties["esp"],3)
    gaus_i.strucC.properties['charge']  += pt.properties["charge"]
    print p,pt.properties["symbol"],pt.properties["charge"]

0 C -0.321
1 C 0.135
2 C -0.237
3 C -0.256
4 S 0.005
5 H 0.241
6 H 0.147
7 H 0.238
8 C -0.234
9 H 0.075
10 H 0.075
11 C 0.259
12 H 0.007
13 H 0.007
14 C -0.414
15 H 0.101
16 H 0.101
17 C -0.026
18 H 0.021
19 H 0.021
20 C 0.204
21 H -0.018
22 H -0.018
23 C -0.443
24 H 0.108
25 H 0.111
26 H 0.108


In [31]:
gaus_i.strucC.calc_charge()
print gaus_i.strucC.charge

-0.003


Clean up charges from rounding

In [33]:
gaus_i.strucC.particles[23].properties["charge"] += 0.001
gaus_i.strucC.particles[20].properties["charge"] += 0.001
gaus_i.strucC.particles[17].properties["charge"] += 0.001

In [34]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()


In [35]:
print gaus_i.strucC.tag

thiophene_hexane


In [37]:
p3ht_n1_esp = buildingblock.Container('thiophene_hexane_esp')

Convert to Bohr to angstroms 

In [38]:
p3ht_n1_esp.convert_gaussian(gaus_i)

In [39]:
p3ht_n1_esp.write_xyz()

In [40]:
p3ht_n1_esp.write_cply()

Build oligomer with length xN

In [41]:
xN = 10

In [42]:
p3ht_xN = buildingblock.Container('p3ht_n%d'%(xN))
p3ht_xN += p3ht_n1_esp

In [43]:
for n in range(1,xN):
    p3ht_xN = buildingblock.attach(p3ht_n1_esp,p3ht_xN,"T",1,"T",0)
    

In [44]:
p3ht_xN.calc_charge()

In [45]:
print p3ht_xN.charge

1.38777878078e-16


In [46]:
p3ht_xN.tag = 'p3ht_n%d_esp'%(xN)

In [47]:
p3ht_xN.write_xyz()

In [48]:
p3ht_xN.write_cply()

In [49]:
del p3ht_xN,p3ht_n1_esp,gaus_i